# Development notebook for optical evaporative cooling DSMC
Patrick Gleeson, Semester 2 2021

In [37]:
# Setup

# using Pkg
# Pkg.add("BenchmarkTools")
# Pkg.add("LinearAlgebra)
# Pkg.add("Calculus")

# using BenchmarkTools
# using Calculus
using LinearAlgebra
using Plots
using Random

# Set random generator seed
Random.seed!(1)

MersenneTwister(1)

## Algorithm

In [38]:
# Distribute atoms

function init_uniform(N, v_th, gas_size)
    # Initialise random positions and velocities in unit cube
    positions = 2 * gas_size * (rand(3, N) :: Matrix{Float64} .- 0.5)
    velocities = 2 * v_th * (rand(3, N) :: Matrix{Float64} .- 0.5)
    active_indices = Vector{Int64}(1:N)

    return positions, velocities, active_indices
end

init_uniform (generic function with 1 method)

In [39]:
# Define a potential and acceleration

function harmonic(wx, wy, wz)
    w_squared = [wx^2, wy^2, wz^2]
    accel(r) = - r .* w_squared
    # Potential _per mass_
    potential(r) = 0.5 * dot(w_squared, r .* r)
    return potential, accel
end

#=
# Defining via automatic differentiation
simplify(differentiate("cos(x)"))
=#

harmonic (generic function with 1 method)

In [40]:
# Apply motion
function free_step!(positions, velocities, dt)
    positions .+= velocities * dt
end

# Velocty Verlet step
function verlet_step!(positions, velocities, accel, dt)
    # Approximate position half-way through time-step
    positions .+= (dt / 2.0) .* velocities
    # Acceleration halfway through timestep.
    # TODO: implement time-dependence, and evaluate at (t + dt/2)
    velocities .+= dt .* hcat([accel(r) for r in eachcol(positions)]...)
    # Updated positions at end of timestep
    positions .+= (dt / 2.0) .* velocities
end

verlet_step! (generic function with 1 method)

In [41]:
# Placeholder collision step
function collisions!(positions, velocities)
    return true
end

collisions! (generic function with 1 method)

In [42]:
# Simulation
function evolve!(positions, velocities, acceleration, potential, duration, dt)
    iterations = convert(Int64, ceil(duration / dt))
    total_ke = zeros(iterations)
    total_pe = zeros(iterations)
    time = zeros(iterations)
    
    t = 0
    i = 1
    while i <= iterations
        # Collisionless motion step
        verlet_step!(positions, velocities, acceleration, dt)
        # Collision step
        collisions!(positions, velocities)
        # Measure energy
        total_ke[i] = ke(velocities)
        total_pe[i] = pe(positions, potential)

        # TODO: measure number density
        # TODO: make potential & acceleration time-dependent
        # Record time
        time[i] = t
        t += dt
        i += 1
    end

    return time, total_ke, total_pe
end

evolve! (generic function with 1 method)

In [43]:
# Measure total energy (per mass)
function ke(velocities::Matrix)
    return 0.5 * sum(velocities .* velocities)
    #sum(velocities .* velocities, dims = 1)
end

function pe(positions::Matrix, potential)
    return sum([potential(r) for r in eachcol(positions)])
end

pe (generic function with 1 method)

## Testing

In [44]:
function test()
    # Simulation parameters
    dt = 1e-4 # timestep
    N = convert(Int64, 1e5) # initial number of atoms
    v_th = 0.01
    gas_size = 10e-6 # approximate size of gas
    duration = 0.1

    # Rough thermal velocity estimate for Rb-87 at 1 microKelvin: 0.01 m/s
    positions, velocities, active_indices = init_uniform(N, v_th, gas_size)
    ω_x = 2π * 150;
    ω_y = 2π * 150;
    ω_z = 2π * 15;
    
    potential, accel = harmonic(ω_x, ω_y, ω_z)
    time, ke, pe = evolve!(positions, velocities, accel, potential, duration, dt)
    plt = plot(time, ke + pe,
         title = "Total energy per atomic mass, over time",
         legend = false)
    xlabel!("Time (s)")
    ylabel!("Massic energy (J/kg)")
    savefig("total-energy.png")
    display(plt)
end

test (generic function with 1 method)

In [45]:
test()